# Typesense

> [Typesense](https://typesense.org) 是一个开源、内存中的搜索引擎，你可以选择 [自行托管](https://typesense.org/docs/guide/install-typesense#option-2-local-machine-self-hosting) 或在 [Typesense Cloud](https://cloud.typesense.org/) 上运行。
>
> Typesense 通过将整个索引存储在 RAM 中（并在磁盘上备份）来专注于性能，同时通过简化可用选项和设置良好默认值来专注于提供开箱即用的开发体验。
>
> 它还允许你将基于属性的过滤与向量查询相结合，以获取最相关的文档。

本笔记本将向您展示如何将 Typesense 用作您的 VectorStore。

让我们先安装所需的依赖：

In [ ]:
%pip install --upgrade --quiet  typesense openapi-schema-pydantic langchain-openai langchain-community tiktoken

我们想要使用 `OpenAIEmbeddings`，所以我们需要获取 OpenAI API 密钥。

In [2]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [6]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Typesense
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

我们将导入我们的测试数据集：

In [19]:
loader = TextLoader("../../how_to/state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

In [ ]:
docsearch = Typesense.from_documents(
    docs,
    embeddings,
    typesense_client_params={
        "host": "localhost",  # Use xxx.a1.typesense.net for Typesense Cloud
        "port": "8108",  # Use 443 for Typesense Cloud
        "protocol": "http",  # Use https for Typesense Cloud
        "typesense_api_key": "xyz",
        "typesense_collection_name": "lang-chain",
    },
)

## 相似性搜索

In [ ]:
query = "What did the president say about Ketanji Brown Jackson"
found_docs = docsearch.similarity_search(query)

In [ ]:
print(found_docs[0].page_content)

## Typesense 作为检索器

Typesense 和所有其他的向量数据库一样，是 LangChain 的一个检索器，使用余弦相似度进行检索。

In [ ]:
retriever = docsearch.as_retriever()
retriever

In [ ]:
query = "What did the president say about Ketanji Brown Jackson"
retriever.invoke(query)[0]